# Run analysis on multiple files

In [20]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import interpolate
import matplotlib.pyplot as plt 
import os
import pyfilaments.analysisutils as analysis
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing

In [22]:
# data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/analysis_test'
data_folder = '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final'

print(os.listdir(data_folder))

# Find all simulation data files and create a list
files_list = []
 # Walk through the folders and identify the simulation data files
for dirs, subdirs, files in os.walk(data_folder, topdown=False):

    root, subFolderName = os.path.split(dirs)

    for fileNames in files:
        if(fileNames.endswith('hdf5') and fileNames[0] != '.' and fileNames == 'SimResults_00.hdf5'):
            files_list.append(os.path.join(dirs,fileNames))

print('Simulation files: ', files_list)

def run_filament_analysis(file):
    print('Analyzing file ...')
    print(file)

    filament = analysis.analysisTools(file = file)


    root_folder, file_name = os.path.split(file)

    # Create a data-packet to save for further analysis
    # Collect simulation metadata
    metadata_path = os.path.join(root_folder, 'metadata.csv')
    assert(os.path.exists(metadata_path))
    df = pd.read_csv(metadata_path)
    
    periodic_flag = None 
    min_period = None
    
    filament.filament_tip_coverage(save = True)
    # Check if simulation was completed or terminated before completion
    if(int(filament.Time[-1]) == int(df[' activity time scale']*500)):
        simulation_completed = True
        # Classify the dynamics
        periodic_flag, min_period = filament.classify_filament_dynamics()
    else:
        simulation_completed = False

    df['simulation completed'] = simulation_completed
    df['periodic dynamics'] = periodic_flag
    df['period'] = min_period
    df['max unique locations'] = filament.derived_data['unique position count'][-1]
    
    save_file = file_name[:-5] + '_analysis.csv'
    save_folder = os.path.join(root_folder, 'Analysis')

    if(not os.path.exists(save_folder)):
        os.makedirs(save_folder)
    df.to_csv(os.path.join(save_folder, save_file))




['activity_time_700', 'activity_time_2000', 'activity_time_900', 'activity_time_600', 'activity_time_1500', 'activity_time_1000', 'activity_time_250', 'activity_time_1250', 'activity_time_750', 'activity_time_500', 'activity_time_1750']
Simulation files:  ['/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final/activity_time_700/SimResults_Np_32_Shape_line_kappa_hat_4.0_k_18.0_b0_2.1_F_0_S_0_D_1.5_activityTime_700_simType_point/SimResults_00.hdf5', '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final/activity_time_700/SimResults_Np_32_Shape_line_kappa_hat_8.0_k_32.0_b0_2.1_F_0_S_0_D_1.5_activityTime_700_simType_point/SimResults_00.hdf5', '/home/deepak/ActiveFilamentsSearch_backup_3/BendingStiffnessSweeps/b0_2_1_FullSweep_Final/activity_time_700/SimResults_Np_32_Shape_line_kappa_hat_10.0_k_40.0_b0_2.1_F_0_S_0_D_1.5_activityTime_700_simType_point/SimResults_00.hdf5', '/home/deepak/ActiveFilamentsSearch_backup_3/Bend

In [23]:
num_cores = multiprocessing.cpu_count()

num_cores = 12

results = Parallel(n_jobs=num_cores,  verbose=10)(delayed(run_filament_analysis)(file) for file in tqdm(files_list))


 35%|███▌      | 48/137 [00:35<01:25,  1.04it/s][Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   40.2s
[Parallel(n_jobs=12)]: Done  37 tasks      | elapsed:   49.0s
100%|██████████| 137/137 [01:41<00:00,  1.35it/s]
[Parallel(n_jobs=12)]: Done 128 out of 137 | elapsed:  1.8min remaining:    7.6s
[Parallel(n_jobs=12)]: Done 137 out of 137 | elapsed:  2.1min finished
